In [117]:

import numpy as np 
import matplotlib.pyplot as pyplot
import pandas as pd
from scipy import ndimage
from PIL import Image



In [118]:
def logisticFunc(x) : 
    return 1./(1.+np.exp(np.negative(x)))

def order1derivative_log(x) :
    return logisticFunc(x)*(1.-logisticFunc(x))



In [119]:
sg_train = Image.open("./data/sg_train.jpg")
sgx_train = Image.open("./data/sgx_train.jpg")
sgx_test = Image.open("./data/sgx_test.jpg")

sg_train_array = np.array(sg_train)
sgx_train_array = np.array(sgx_train)
sgx_test_array = np.array(sgx_test)

#print(sgx_train.mode)# 'L'
#pyplot.subplot(3,1,1)
#pyplot.imshow(sg_train, cmap=pyplot.gray())# mode = 'L' which means gray scale
#pyplot.subplot(3,1,2)
#pyplot.imshow(sgx_train, cmap=pyplot.gray())# mode = 'L' which means gray scale
#pyplot.subplot(3,1,3)
#pyplot.imshow(sgx_test, cmap=pyplot.gray())# mode = 'L' which means gray scale
#pyplot.show()



In [120]:

def generate_patches(img,patchsize):
    patch_mat = None
    for row in range(0,200-patchsize+1):
        for col in range(0,200-patchsize+1):
            if patch_mat is None : 
                patch_mat = img[row:row+patchsize, col : col+patchsize].flatten()
                #patch_mat = patch_mat.flatten() 
                continue
                
                
            a = img[row:row+patchsize,col:col+patchsize]
            a = a.flatten()
            #print(row,col,a.shape, patch_mat.size)
            patch_mat = np.vstack((patch_mat,a))
    return patch_mat.T

In [121]:
sgx_train_patchmat = generate_patches(sgx_train_array,filter_Size)

In [122]:
sg_train_shape=sg_train_array.shape

In [123]:
filter_size=11#taking a wondow of size 11
trim_start_ind= int((filter_size-1)/2 )
trim_stop_ind=sg_train_shape[0]-int((filter_size-1)/2)
trimmed_sg_train=sg_train_array[trim_start_ind:trim_stop_ind,trim_start_ind:trim_stop_ind]
trimmed_sg_train.shape

(190, 190)

In [124]:
trimmed_sg_train = trimmed_sg_train.flatten()
trimmed_sg_train = trimmed_sg_train.reshape((1,len(trimmed_sg_train)))

In [125]:
image_filter=np.random.randn(filter_size,filter_size)
image_filter=image_filter.flatten()
#imageRavel=imageFilter.ravel()
print(image_filter.min(), image_filter.flatten().max())
print("sg_train shape: ",sg_train_array.shape," Filter shape: ",image_filter.shape)

-2.01178821457 1.86832181578
sg_train shape:  (200, 200)  Filter shape:  (121,)


In [126]:
image_filter = image_filter.reshape((filter_size**2,1))

In [127]:
sgx_train_array = sgx_train_patchmat/255.0 #normalizing the pixel intensity

In [128]:
trimmed_sg_train = trimmed_sg_train/255.0 #normalising the pixel intensity

In [129]:
rate=0.001
error=1000
threshold = 1e-10
iter=0
while error > threshold :
    
    sg_train_image = image_filter.T.dot(sgx_train_patchmat)
    diff = trimmed_sg_train - logisticFunc(sg_train_image)
    
    error = diff.dot(diff.T)/trimmed_sg_train.shape[1]
    
    gradient = float(2)/36100.0 * sgx_train_patchmat.dot((diff * order1derivative_log(sg_train_image)).T)

    imageFilter = image_filter + rate * gradient
    print(iterations, error)
    if iterations % 1000 == 0 : 
        print(iter, error)
    iter +=1
    if iter > 5 : 
        break

/home/sakorrap/.virtualenvs/sah/lib/python3.5/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


6 [[ 0.36240671]]
6 [[ 0.36240671]]
6 [[ 0.36240671]]
6 [[ 0.36240671]]
6 [[ 0.36240671]]
6 [[ 0.36240671]]
